# SageMaker Experimentsを最小構成で実行する

* Amazon SageMaker Experiments Python SDK doc
* https://sagemaker-experiments.readthedocs.io/en/latest/#

* SageMaker Experiments Python SDK
* https://github.com/aws/sagemaker-experiments

* GitHub samples
* https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker-experiments

* Developer Guide
* https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html

SageMaker Studio の Python 3(PyTorch 1.10 Python 3.8 CPU Optimized)で動作確認

In [ ]:
!pip install ipywidgets widgetsnbextension # studioの場合は必要。要カーネル再起動

* torch / torchvision : 1.11.0 / 0.5.0 -> NG
* torch / torchvision : 1.10.0 / 0.5.0 -> NG
* torch / torchvision : 1.5.0 / 0.5.0 -> OK
* torch / torchvision : 1.7.0 / 0.5.0 -> OK
* torch / torchvision : 1.9.0 / 0.10.0 -> NG
* torch / torchvision : 1.10.2 / 0.11.3 -> NG
* torch / torchvision : 1.9.1 / 0.10.1 -> NG
* torch / torchvision : 1.8.0 / 0.9.0 -> OK

In [ ]:
!pip install -U torch==1.8.0 # 1.4.0で動作確認済み # 1.11.0 NG

In [ ]:
!pip install torchvision==0.9.0 # 0.5.0で動作確認済み

In [ ]:
!pip freeze | grep torch

In [ ]:
from torchvision.datasets import MNIST
from torchvision import transforms

MNIST.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/MNIST/"]

MNIST(
    'data',
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
)

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist-wandb-studio'

role = sagemaker.get_execution_role()

In [ ]:
sagemaker.__version__

In [ ]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

In [ ]:
inputs

In [ ]:
!pygmentize src/mnist-experiments-tracker.py

### estimatorを宣言

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='mnist-experiments-tracker.py',
                    source_dir="src",
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=1,
                    instance_type='ml.c5.2xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'backend': 'gloo'
                    },
                    metric_definitions=[
                        {'Name': 'training/loss', 'Regex': 'Loss: (.*?)#015'},
                        {'Name': 'testing/loss', 'Regex': 'Average loss: (.*?),'},
                    ])

## experiments, trisal, trial_component, trackerを宣言

In [ ]:
from smexperiments import experiment, tracker

my_experiment = experiment.Experiment.create(experiment_name='myExperiment')
my_trial = my_experiment.create_trial(trial_name='job001')

In [ ]:
# use `with` statement to ensure `my_tracker.close()` is called
with tracker.Tracker.create(display_name='training') as my_tracker:
    my_tracker.log_input(name="input-dataset-dir", media_type="s3/uri", value=inputs)

### 学習実施時に、作成したコンポーネントを指定

In [ ]:
# 確認
my_trial.trial_name

In [ ]:
# 確認
my_tracker.trial_component.display_name

In [ ]:
estimator.fit({'training': inputs},
    # trialの情報を指定
    experiment_config={
        #"ExperimentName": my_experiment.experiment_name,
        "TrialName": my_trial.trial_name,
        "TrialComponentDisplayName": my_tracker.trial_component.display_name,
    })